Importamos los datos de la base de datos de `Khan`, esta es una base de datos donde se vienen 83 muestras y 2308 variables que consisten en la expresión génica estandarizada de distintos genes. La variable de salida cuenta con valores numéricos del 1 al 4 que
corresponden a distintos tipos de cáncer, o sea, una clase. Antes de empezar a trabajar con este dataframe revisamos que no tenga huecos, a lo que no se encuentra ninguno. Posteriormente, buscamos comparar las clases 2 y 4, por lo que asignamos la salida a cada grupo como grupo2 como aquellas cuya salida Y sea 2 y grupo4 para cuyas salidas Y sea 4. Despues, tomamos todos los valores dentro de una variable cuyas observaciones hayan tenido como salida 2 y lo promediamos, a continuacion hacemos lo mismo para salidas de 4. Despues el promedio para cada variable, buscamos la diferencia entre las que tenian como salida 2 y 4 para luego enlistar las 10 de diferencia mas grande. Estas diferencias podrian ayudar a distinguir entre los tipos de cáncer y de manera que a mayor diferencia, se podria interpretar que "el comportamiento del gen varia dependiendo del tipo de cancer".

In [37]:
import pandas as pd

df = pd.read_csv("A3.1 Khan.csv")

print("Valores faltantes por columna:")
print(df.isnull().sum().sum())
print()

grupo2 = df[df["y"] == 2]
grupo4 = df[df["y"] == 4]

diferencia = (grupo2.mean(numeric_only=True) - grupo4.mean(numeric_only=True)).abs()

top10 = diferencia.sort_values(ascending=False).head(10)

print(top10)

Valores faltantes por columna:
0

X187     3.323151
X509     2.906537
X2046    2.424515
X2050    2.401783
X129     2.165185
X1645    2.065460
X1319    2.045941
X1955    2.037340
X1003    2.011337
y        2.000000
dtype: float64


En este análisis, se calcularon pruebas t para comparar las medias de expresión de cada gen entre las clases 2 y 4.
Sin embargo, dado que se realizaron más de dos mil comparaciones (una por cada gen), existe una alta probabilidad de encontrar falsos positivo, o sea, genes que parecen diferir entre clases solo por azar. Por esta razón, aplicamos diferentes métodos de corrección por comparaciones múltiples, que ajustan los valores p para mantener control sobre los errores estadísticos.

Método de Bonferroni:
Este método controla el error familiar (FWER), es decir, la probabilidad de cometer al menos un error tipo I en todas las pruebas. Se logra dividiendo el nivel de significancia α entre el número total de pruebas (m), o multiplicando cada p-value por m. En la práctica, esto hace que el criterio para rechazar una hipótesis sea mucho más estricto 
$$
p_i < \frac{\alpha}{m}
$$
por lo que rara vez se declara algo como significativo. En nuestro contexto, Bonferroni reduce casi completamente los falsos positivos, pero también puede eliminar genes que realmente sí difieren entre clases.

Método de Holm-Bonferroni:
El método de Holm es una versión más flexible de Bonferroni, ya que el umbral de rechazo depende de la posición ordenada de cada p-value. Primero se ordenan los valores p de menor a mayor, y luego se compara cada uno con un límite 
$$
p_{(j)} < \frac{\alpha}{m + 1 - j}
$$ 
donde j es su posición en la lista. Esto significa que los p-values más pequeños tienen un umbral más estricto, y los más grandes, más flexible.

Método de Benjamini-Hochberg (FDR):
Finalmente, el método de Benjamini-Hochberg no controla el FWER, sino la tasa de falsos descubrimientos (FDR).
En lugar de intentar evitar cualquier falso positivo, tolera que exista un pequeño porcentaje de ellos, pero asegura que su proporción sea baja.
Esto lo hace ideal para estudios genómicos como este, donde se analizan miles de genes. El método ordena los p-values y los compara con umbrales proporcionales a su posición
$$
p_{(j)} < \frac{q \times j}{m}
$$
garantizando que, por ejemplo, si el FDR se fija en 0.05, no más del 5% de los genes detectados como significativos son falsos positivos en promedio. Por ello, Benjamini-Hochberg suele ser el método más apropiado en análisis de expresión génica masiva.


In [38]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

genes = [col for col in df.columns if col != "y"]
t_stats = []
p_values = []

for gen in genes:
    t, p = ttest_ind(grupo2[gen], grupo4[gen], equal_var=False) 
    t_stats.append(t)
    p_values.append(p)

resultados = pd.DataFrame({
    "Gen": genes,
    "t_stat": t_stats,
    "p_value": p_values
})

# Bonferroni
resultados["p_bonf"] = multipletests(resultados["p_value"], method='bonferroni')[1]
# Holm
resultados["p_holm"] = multipletests(resultados["p_value"], method='holm')[1]
# Benjamini-Hochberg (FDR)
resultados["p_bh"] = multipletests(resultados["p_value"], method='fdr_bh')[1]

#significativos (α = 0.05)
resultados["sig_bonf"] = resultados["p_bonf"] < 0.05
resultados["sig_holm"] = resultados["p_holm"] < 0.05
resultados["sig_bh"] = resultados["p_bh"] < 0.05

signif_bonf = resultados[resultados["sig_bonf"]]
signif_holm = resultados[resultados["sig_holm"]]
signif_bh = resultados[resultados["sig_bh"]]

print("Bonferroni:", len(signif_bonf), "genes significativos")
print("Holm:", len(signif_holm), "genes significativos")
print("Benjamini-Hochberg:", len(signif_bh), "genes significativos")

print("\nGenes significativos con Bonferroni:\n", signif_bonf.head())
print("\nGenes significativos con Holm:\n", signif_holm.head())
print("\nGenes significativos con Benjamini-Hochberg:\n", signif_bh.head())


Bonferroni: 72 genes significativos
Holm: 72 genes significativos
Benjamini-Hochberg: 296 genes significativos

Genes significativos con Bonferroni:
       Gen    t_stat       p_value    p_bonf    p_holm          p_bh  sig_bonf  \
1      X2 -6.900138  7.383962e-08  0.000170  0.000168  5.497479e-06      True   
35    X36  5.610781  7.885432e-07  0.001820  0.001786  4.136268e-05      True   
66    X67 -4.793322  1.413077e-05  0.032614  0.031681  4.867735e-04      True   
128  X129 -8.412602  2.516574e-09  0.000006  0.000006  2.904126e-07      True   
173  X174 -6.974367  5.603441e-09  0.000013  0.000013  6.158448e-07      True   

     sig_holm  sig_bh  
1        True    True  
35       True    True  
66       True    True  
128      True    True  
173      True    True  

Genes significativos con Holm:
       Gen    t_stat       p_value    p_bonf    p_holm          p_bh  sig_bonf  \
1      X2 -6.900138  7.383962e-08  0.000170  0.000168  5.497479e-06      True   
35    X36  5.610781  7.8

Para hacer la comparativa, ahora se realizó un análisis de varianza (ANOVA) para comparar las medias de expresión génica entre las cuatro clases de cáncer presentes en la base de datos Khan. El ANOVA evalúa si las diferencias entre los promedios de las clases son estadísticamente significativas. Un valor p bajo (< 0.05) indica que al menos una clase presenta una expresión distinta para ese gen. Tras aplicar la corrección por múltiples comparaciones mediante el método de Benjamini-Hochberg, se identificaron los genes cuya expresión difiere significativamente entre las cuatro clases. Estos genes son los que más probablemente participen en los mecanismos biológicos que distinguen los distintos tipos de cáncer, y podrían representar marcadores potenciales para estudios posteriores de clasificación o inferencia biológica.

In [39]:
from scipy.stats import f_oneway
from statsmodels.stats.multitest import multipletests

# clases
grupo1 = df[df["y"] == 1]
grupo2 = df[df["y"] == 2]
grupo3 = df[df["y"] == 3]
grupo4 = df[df["y"] == 4]

#ANOVA
genes = [col for col in df.columns if col != "y"]
f_stats = []
p_values = []

for gen in genes:
    f, p = f_oneway(grupo1[gen], grupo2[gen], grupo3[gen], grupo4[gen])
    f_stats.append(f)
    p_values.append(p)

resultados_anova = pd.DataFrame({
    "Gen": genes,
    "F_stat": f_stats,
    "p_value": p_values
})

#FDR Benjamini-Hochberg
resultados_anova["p_bh"] = multipletests(resultados_anova["p_value"], method='fdr_bh')[1]
resultados_anova["significativo"] = resultados_anova["p_bh"] < 0.05

signif_anova = resultados_anova[resultados_anova["significativo"]]
print("Número de genes con diferencias significativas entre las 4 clases:", len(signif_anova))
print(signif_anova.sort_values("p_bh").head(10))


Número de genes con diferencias significativas entre las 4 clases: 1162
        Gen     F_stat       p_value          p_bh  significativo
1954  X1955  84.364086  1.459035e-24  2.045755e-21           True
1388  X1389  83.817537  1.772751e-24  2.045755e-21           True
1002  X1003  77.795622  1.618988e-23  1.245542e-20           True
2049  X2050  69.230799  4.733702e-22  2.731346e-19           True
245    X246  68.414042  6.633722e-22  3.062126e-19           True
741    X742  65.572797  2.195548e-21  8.445542e-19           True
0        X1  59.118264  3.839240e-20  1.265852e-17           True
2161  X2162  56.987623  1.035143e-19  2.986387e-17           True
1953  X1954  55.419914  2.182635e-19  5.597246e-17           True
186    X187  54.615724  3.217900e-19  6.751740e-17           True


Se separaron los datos en conjuntos de entrenamiento (70%) y prueba (30%).
A partir de los genes con mayor diferencia de expresión entre clases, se entrenaron tres modelos de Máquinas de Vectores de Soporte (SVM) con distintos tipos de kernel: lineal, polinomial de grado 3 y radial.

Aunque la selección de variables se realizó con base en el análisis previo (usando todos los datos), esto implica una fuga de datos, ya que la información del conjunto de prueba influyó indirectamente en la elección de características. En un análisis riguroso, la selección de genes debería realizarse solo con los datos de entrenamiento, pero por esta ocacion se pasara por alto.

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# genes dados por el AVOVA
genes_seleccionados = [
    "X1955", "X1389", "X1003", "X2050", "X246",
    "X742", "X1", "X2162", "X1954", "X187"
]

X = df[genes_seleccionados]
y = df["y"]

# train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# entrenar SVM 
svm_lineal = SVC(kernel='linear', random_state=42)
svm_poly = SVC(kernel='poly', degree=3, random_state=42)
svm_rbf = SVC(kernel='rbf', random_state=42)

svm_lineal.fit(X_train, y_train)
svm_poly.fit(X_train, y_train)
svm_rbf.fit(X_train, y_train)

modelos = {
    "Lineal": svm_lineal,
    "Polinomial (grado 3)": svm_poly,
    "Radial (RBF)": svm_rbf
}

Después de entrenar los tres modelos SVM (lineal, polinomial y radial), se calcularon distintas métricas de desempeño con el fin de comparar su rendimiento.
Para ello, se utilizaron las funciones accuracy_score, precision_score, recall_score, f1_score y confusion_matrix de la librería scikit-learn.

El código recorre los tres modelos almacenados en el diccionario modelos, genera las predicciones correspondientes (y_pred) sobre el conjunto de prueba y calcula las métricas principales:
Exactitud (accuracy): mide la proporción total de aciertos del modelo.
Precisión: indica qué proporción de las predicciones positivas fueron correctas.
Recall (sensibilidad): muestra qué proporción de los casos reales fueron detectados correctamente.
F1-Score: representa el equilibrio entre la precisión y el recall, útil para evaluar modelos con clases desbalanceadas.

Se utiliza el parámetro average='weighted' para que las métricas se calculen considerando el tamaño (support) de cada clase.
Los resultados se almacenan en una lista de diccionarios y se convierten en un DataFrame (tabla_resultados) para visualizar de manera comparativa el desempeño de los tres modelos.

Finalmente, se imprime la matriz de confusión de cada modelo. Esta matriz permite observar el número de aciertos y errores por clase, identificando qué clases son más difíciles de distinguir, ademas de que al estar trabajando con todas las clases (1, 2, 3, 4) se espera verificar cuales de las salidas son desplegadas correctamente y cuales fueron predichas incorrectamente. La forma en la que esta acomodada es en orden ascendente, desde 1 hasta 4 de izquierda a derecha siendo lo predicho y de 1 a 4 de arriba a abajo siendo esto lo el dato real.

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

metricas = []

for nombre, modelo in modelos.items():
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    metricas.append({
        "Modelo": nombre,
        "Exactitud": acc,
        "Precisión": prec,
        "Recall": rec,
        "F1-Score": f1
    })

tabla_resultados = pd.DataFrame(metricas)
print(tabla_resultados)

for nombre, modelo in modelos.items():
    print(f"\nMatriz de confusión - {nombre}")
    print(confusion_matrix(y_test, modelo.predict(X_test)))


                 Modelo  Exactitud  Precisión  Recall  F1-Score
0                Lineal       0.92   0.930000    0.92  0.921524
1  Polinomial (grado 3)       0.88   0.912727    0.88  0.878297
2          Radial (RBF)       0.96   0.964000    0.96  0.959719

Matriz de confusión - Lineal
[[3 0 0 0]
 [1 8 0 0]
 [0 0 5 0]
 [0 1 0 7]]

Matriz de confusión - Polinomial (grado 3)
[[3 0 0 0]
 [0 8 0 1]
 [0 0 3 2]
 [0 0 0 8]]

Matriz de confusión - Radial (RBF)
[[3 0 0 0]
 [0 9 0 0]
 [0 0 5 0]
 [0 1 0 7]]
